# Домашнее задание 1

NB! Важный дисклеймер, который нужно учитывать во всех домашних заданиях по курсу. Обосновывайте ваши действия. При проверке нам важно увидеть, что вы комплексно подходите к задаче и понимаете, зачем нужны те или иные шаги. Примеры важных моментов:

* Я использую следующие предпосылки: ...
* Я использую признак X, потому что ...
* Я выбираю модель Y с такими-то параметрами, потому что ...
* Я использую для входных данных такое-то преобразование, потому что характеристики ряда XXX, а я хочу добиться YYY.
* Я использую периоды сезонности X и Y, потому что вижу на графиках ...
* ...

Необоснованные дейтвия могут остаться без оценки.

В этом домашнем задании мы немного поупражняемся с обработкой временных данных и ML-моделями. 

Мы сотрудники одного из крупных банков с развитой сетью банкоматов. Предположим, что мы рассматриваем банкоматы, которые работают только на выдачу наличных. Возможно на текущий момент это редкость, но для периода, когда собирался датасет, они были распространены.

Наша задача состоит в том, чтобы предсказать спрос на наличность и вовремя загрузить в банкомат деньги, чтобы он работал бесперебойно. В то же время, мы хотим не гонять доблестных инкассаторов к наполовину полным банкоматам, так как их перемещения стоят денег. Попытаемся построить модель, которая для конкретного банкомата будет прогнозировать количество денег, которые будет необходимо выдать в следующие периоды. 

Конечно, в оригинале задача была сложнее, так как банкоматы могут ломаться и простаивать, о чём вендору известно. Ещё, например, некоторые банкоматы доступны 24 часа в сутки, а некоторые -- наоборот, стоят в закрываемых помещениях. Но мы на такой уровень погружаться не будем.

В файле `data.csv` представлены три временных ряда. Каждый из них представляет собой объём выданной наличности в определённом банкомате. 

В файле `description.csv` представлено краткое описание каждого банкомата: ID, Адрес, описание местоположения, а также географические координаты. 

Каждый из вас будет использовать только один ряд. Жеребьёвка будет проводится с помощью сложнейшего стохастического алгоритма в ячейке ниже.

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.decomposition import PCA
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from statsmodels.tsa.seasonal import STL, MSTL

import optuna
from sklearn.metrics.pairwise import rbf_kernel as RBF
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import acf, pacf
from pandas.tseries.offsets import CustomBusinessDay
from sktime.transformations.series.holiday import CountryHolidaysTransformer
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from functools import reduce
from IPython.display import clear_output
import time
import ephem
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone

In [4]:
full_name = "Добин Илья Николаевич" #Введите ваше ФИО

print(f"ID вашего временного ряда: {len(full_name)%3}")

ID вашего временного ряда: 0


## 1) Загрузка данных (0.5 балла)

Загрузите временной ряд. Преобразуйте даты к Timestamp. Определите частоту данных. 

* Если необходимо, исходя из частоты выберите [оффсет](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) и ресэмплируйте к нему.
* Если необходимо, заполните пропуски
* Если необходимо, сделайте иные преобразования ряда

In [30]:
import pandas as pd
dt = pd.read_csv('data.csv')
dt.Date = pd.to_datetime(dt.Date)
dt.set_index('Date', inplace=True)

offst = 'D'
dt_r = dt.resample(offst).sum()
dt_r = dt_r.rename(columns={"0": "Target"})

## 2) Эксплоративный анализ (0.75 балла)

#### 2.1) Визуальный анализ (0.5 балла)

Изобразите временной ряд и его автокорреляции до порядка, который сочтёте нужным.

Что вы можете сказать о структуре ряда? Присутствует ли в нём тренд? Сезонность? Цикличность? Как эти особенности могут помочь прогнозировать?  Ответьте на **каждый** вопрос. Можете дополнительно построить

`Hint`: Для трендированных рядов: если построить коррелограммы на приращениях ряда, а не на исходных величинах, то некоторые паттерны видны лучше.


In [34]:
# ༼ つ ◕_◕ ༽つ
def plot_ts_anal(time_series_list, lags=30, show_ts=True, show_pacf=False, show_acf=False, show_diff=False, show_diff_pacf=False, show_diff_acf=False, one=False):
    if one:
        fig = go.Figure()
        for ts in time_series_list:
            dt_r = ts['data']
            ts_name = ts['name']
            if show_ts:
                fig.add_trace(go.Scatter(x=dt_r.index, y=dt_r, mode='lines+markers', name=f'TS: {ts_name}'))
        fig.update_layout(title_text="Combined Time Series", showlegend=True, width=1200, height=600)
        fig.show()
    else:
        analyses_count = sum([show_ts, show_pacf, show_acf, show_diff, show_diff_pacf, show_diff_acf])
        total_plots = analyses_count * len(time_series_list)

        subplot_titles = []
        for ts in time_series_list:
            ts_name = ts['name']
            if show_ts:
                subplot_titles.append(f'Time Series: {ts_name}')
            if show_pacf:
                subplot_titles.append(f'PACF: {ts_name}')
            if show_acf:
                subplot_titles.append(f'ACF: {ts_name}')
            if show_diff:
                subplot_titles.append(f'Difference: {ts_name}')
            if show_diff_pacf:
                subplot_titles.append(f'Difference PACF: {ts_name}')
            if show_diff_acf:
                subplot_titles.append(f'Difference ACF: {ts_name}')

        fig = make_subplots(rows=total_plots, cols=1, shared_xaxes=True, vertical_spacing=0.02, subplot_titles=subplot_titles)

        current_row = 1
        for ts in time_series_list:
            dt_r = ts['data']
            ts_name = ts['name']

            if show_ts:
                fig.add_trace(go.Scatter(x=dt_r.index, y=dt_r, mode='lines+markers', name=f'TS: {ts_name}'), row=current_row, col=1)
                current_row += 1

            if show_pacf:
                pacf_values = pacf(dt_r, nlags=lags, method='ywmle')
                for i, value in enumerate(pacf_values):
                    fig.add_trace(go.Scatter(x=[i, i], y=[0, value], mode='lines', showlegend=False), row=current_row, col=1)
                fig.add_trace(go.Scatter(x=list(range(len(pacf_values))), y=pacf_values, mode='markers', name=f'PACF: {ts_name}'), row=current_row, col=1)
                current_row += 1

            if show_acf:
                acf_values = acf(dt_r, nlags=lags, fft=True)
                for i, value in enumerate(acf_values):
                    fig.add_trace(go.Scatter(x=[i, i], y=[0, value], mode='lines', showlegend=False), row=current_row, col=1)
                fig.add_trace(go.Scatter(x=list(range(len(acf_values))), y=acf_values, mode='markers', name=f'ACF: {ts_name}'), row=current_row, col=1)
                current_row += 1

            if show_diff:
                diff = dt_r.diff().dropna()
                fig.add_trace(go.Scatter(x=diff.index, y=diff, mode='lines+markers', name=f'Diff: {ts_name}'), row=current_row, col=1)
                current_row += 1

            if show_diff_pacf:
                if show_diff:
                    diff_pacf_values = pacf(diff, nlags=lags, method='ywmle')
                    for i, value in enumerate(diff_pacf_values):
                        fig.add_trace(go.Scatter(x=[i, i], y=[0, value], mode='lines', showlegend=False), row=current_row, col=1)
                    fig.add_trace(go.Scatter(x=list(range(len(diff_pacf_values))), y=diff_pacf_values, mode='markers', name=f'PACF Diff: {ts_name}'), row=current_row, col=1)
                    current_row += 1

            if show_diff_acf:
                if show_diff:
                    diff_acf_values = acf(diff, nlags=lags, fft=True)
                    for i, value in enumerate(diff_acf_values):
                        fig.add_trace(go.Scatter(x=[i, i], y=[0, value], mode='lines', showlegend=False), row=current_row, col=1)
                    fig.add_trace(go.Scatter(x=list(range(len(diff_acf_values))), y=diff_acf_values, mode='markers', name=f'ACF Diff: {ts_name}'), row=current_row, col=1)
                    current_row += 1

        fig.update_layout(
                title_text="Time Series Analysis", 
                showlegend=True,
                width=1200, 
                height=200 * total_plots,
                margin=dict(l=20, r=20, t=40, b=20) 
            )
        fig.show()



In [36]:
plot_ts_anal([{'data': dt_r['Target'], 'name': 'Банкоматы'}], lags=100, show_ts=True, show_pacf=True, show_acf=True, show_diff=True, show_diff_pacf=True, show_diff_acf=True)


Вывод: вот такие вот дела!

#### 2.2) STL/MSTL (0.25 балла)

Определите периоды сезонности, которые содержатся в ряде. Выполните STL/MSTL. Желательно, чтобы в тренде и остатках не содержалось явных сезонных колебаний. Чем можно объяснить каждый из периодов сезонности?

In [46]:
# ༼ つ ◕_◕ ༽つ
stl = STL(dt_r['Target'].diff().dropna(),trend=365, seasonal = 5)
radi = stl.fit()

In [47]:
plot_ts_anal(
    [{'data': radi.trend, 'name': 'Тренд'}, 
     {'data': radi.seasonal, 'name': 'Сезонность'}, 
     {'data': radi.resid, 'name': 'Остатки'}, 
     {'data': radi.observed, 'name': 'Исходный ряд'}],
    show_ts=True, 
    show_pacf=False, 
    show_acf=False, 
    show_diff=False, 
    show_diff_pacf=False, 
    show_diff_acf=False, 
    lags=100)


Вывод: Жесть!

In [48]:
dt_r_2 = dt_r

## 3) Выбор признаков (1.75 балла)

#### 3.1) Стандартные признаки (1.25 балла)
Исходя из свойств ряда, а также дополнительной информации из файла `description.csv` определите список фич, которые вы будете использовать для прогнозирования.

Мы подробно обсуждали генерацию признаков на семинаре. У вас есть следующие опции:

1) Использование времени (индикаторы, сезонность через тригонометрию, структурные сдвиги, календари праздников и т.п.)
2) Лаги таргета и различные функции от них
3) Статистики, рассчитанные по окну

Обоснуйте выбор каждой переменной. 

Требования: 
* Не менее 10 признаков
* Обоснование признаков должно согласовываться с характеристиками ряда из пункта 2 и здравым смыслом.
* За каждую логическую ошибку в обосновании или отсутствие обоснования снимается 0.5 балла (макс. 1.5 балла, в минус не уходим)

#### 3.2) Внешние переменные (0.5 балла)
Придумайте какую-нибудь интересную переменную и найдите данные для неё в интернете, исходя из местоположения банкомата и его описания. Обоснуйте её значимость. 

Бонус до 0.5 балла: На усмотрение ассистента, если было собрано несколько достойных переменных или было сделано что-то неординарное с вау-эффектом. 

**1. Календарные признаки**

In [56]:
# ༼ つ ◕_◕ ༽つ
holidays = CountryHolidaysTransformer("RUS").fit_transform(dt_r_2)  
dt_r_2['weekday'] = dt_r_2.index.weekday
dt_r_2['month'] = dt_r_2.index.month
dt_r_2['day_of_month'] = dt_r_2.index.day
dt_r_2['quarter'] = dt_r_2.index.quarter
dt_r_2['is_holiday'] = holidays['RUS_holidays'].values
dt_r_2['is_holiday'] = dt_r_2['is_holiday'].replace({True:1,False:0})
dt_r_2['MA7'] = dt_r_2.Target.shift().rolling(7).mean()
dt_r_2['MA31'] = dt_r_2.Target.shift().rolling(31).mean()

Обоснование: логично!
![](lol.png)

**2. Лаги**

In [57]:
dt_r_2['lag_1'],  dt_r_2['lag_7'], dt_r_2['lag_30'], dt_r_2['lag_90'], dt_r_2['lag_365'] = dt_r_2['Target'].shift(1), dt_r_2['Target'].shift(5), dt_r_2['Target'].shift(22), dt_r_2['Target'].shift(65), dt_r_2['Target'].shift(365)
dt_r_2 = dt_r_2.fillna(0)

Обоснование: логично!
![](lol.png)

In [61]:
result = dt_r_2.fillna(0)

In [62]:
result = result.apply(pd.to_numeric, errors='coerce')

In [63]:
result.to_csv('featured_Data.csv')

## 4) Одношаговое прогнозирование (2.2 балла)

#### 4.1) Предобработка (0.5 балла)

Сформируйте таблицу для одношагового прогнозирования. 

* Разбейте полученные данные на трейн, валидацию и тест. Размеры выборок: 0.6, 0.2 и 0.2 соответственно. Не используйте шаффл, разбивайте упорядоченную выборку.
* Если необходимо, отнормируйте данные. Не забудьте, что валидацию или тест не надо нормировать. 
* Если необходимо, закодируйте категориальные фичи любым валидным методом.

Во временных рядах очень важно избегать утечек в данных (data leak). Их легко допустить при неаккуратной генерации признаков. Например, если вы для прогноза $y_t$ будете исплользовать фичу разности $y_t - y_{t-1}$. Будьте осторожны и внимательны.


In [70]:
# ༼ つ ◕_◕ ༽つ
ts = pd.read_csv('featured_Data.csv', index_col=False)
ts = ts.drop(ts.columns[0], axis = 1)
ts = ts.drop(ts.columns[0], axis = 1)
ts = ts.drop(ts.columns[1], axis = 1)
ts

,Target,weekday,month,day_of_month,quarter,is_holiday,MA7,MA31,lag_1,lag_7,lag_30,lag_90,lag_365
0,0.0,3,1,1,1,1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,4,1,2,1,1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,5,1,3,1,1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,6,1,4,1,1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0,1,5,1,1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,56700.0,2,9,13,3,0,155114.285714,91990.322581,226200.0,135000.0,15000.0,66600.0,122400.0
987,209500.0,3,9,14,3,0,130485.714286,93819.354839,56700.0,1000.0,68600.0,100000.0,248900.0
988,346700.0,4,9,15,3,0,146242.857143,100416.129032,209500.0,0.0,30700.0,89300.0,271500.0
989,29200.0,5,9,16,3,0,176485.714286,108900.000000,346700.0,395300.0,52800.0,154300.0,243800.0


In [71]:
train_size = int(len(ts) * 0.8)
X, y = ts.drop('Target', axis=1), ts['Target']

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [72]:
X_train

,weekday,month,day_of_month,quarter,is_holiday,MA7,MA31,lag_1,lag_7,lag_30,lag_90,lag_365
0,3,1,1,1,1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,4,1,2,1,1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,5,1,3,1,1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,6,1,4,1,1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,0,1,5,1,1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
787,6,2,26,1,0,152400.000000,180280.645161,58200.0,316500.0,3500.0,322800.0,72300.0
788,0,2,27,1,0,152400.000000,179067.741935,0.0,351000.0,1200.0,42300.0,2500.0
789,1,2,28,1,0,129728.571429,183609.677419,172400.0,10000.0,41100.0,1500.0,441700.0
790,2,3,1,1,0,153828.571429,198870.967742,485200.0,0.0,324100.0,521200.0,193400.0


#### 4.2) Регрессия (1 балл)

Выберите вашу любимую модель и решите задачу одношагового прогнозирования. 

* Подберите гиперпараметры на валидационной выборке.
* После подбора гиперпараметров обучите ваш регрессор на трейне + валидации.

 Кросс-валидацию мы ещё пройти не успели, поэтому метрику будем считать только по тестовой выборке. Подсчитайте качество одношаговых прогнозов. В качестве метрики используйте WAPE. 

$$WAPE=\frac{\sum_{t=1}^n\left|A_t-F_t\right|}{\sum_{t=1}^n\left|A_t\right|}, \quad A_t - Actual, F_t - Forecast$$

При подсчёте метрик вам нужно построить прогнозы в номированных величинах, использовать параметры тренировочноый (или тренировочной + валидационной) выборки для обратной нормировки, а потом подсчитать метрики в исходных величинах. Все метрики вычисляются в исходных величинах!

In [74]:
# ༼ つ ◕_◕ ༽つ
def cos(x):
    return np.cos(x)

def wape(y_test, y_pred):
    return np.sum(np.abs(y_test - y_pred)) / np.sum(np.abs(y_test))

class RFFPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, function = cos, n_features=1000, new_dim=50, use_PCA=True, classifier='logreg', lambd = 1):
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.new_dim = new_dim
        self.classifier = classifier
        self.function = function
        self.lambd = lambd
    def fit(self, X, y):
        if self.use_PCA == True:
            self.pca = PCA(n_components=self.new_dim, whiten=True)
            new_x = self.pca.fit_transform(X)
        else:
            new_x = X
            
        i = np.random.randint(X.shape[0], size=self.n_features)
        j = np.random.randint(X.shape[0], size=self.n_features)
        self.sigma = np.median(np.sum((new_x[i, :] - new_x[j, :])**2, axis=1))
        self.w, self.b = np.random.normal(loc=0, scale = 1 / np.sqrt(self.sigma), size=(new_x.shape[1], self.n_features)), np.random.uniform(low=-np.pi, high=np.pi, size= self.n_features)
        self.new_features = self.function(new_x @ self.w + self.b)

        if self.classifier == 'logreg':
            self.model = LogisticRegression(max_iter=100)
            self.model.fit(self.new_features, y)
        if self.classifier == 'SVM':
            self.model = SVC(kernel='linear',probability=True)
            self.model.fit(self.new_features, y)
        if self.classifier == 'regression':
            self.model = Ridge(self.lambd)
            self.model.fit(self.new_features, y)
        return self
        

    def predict_proba(self, X):
        if self.use_PCA == True:
            X = self.pca.transform(X)      
        new_x_test = self.function(X @ self.w + self.b)

        return self.model.predict_proba(new_x_test)
        
    def predict(self, X):
        if self.use_PCA == True:
            X = self.pca.transform(X)     
        new_x_test = self.function(X @ self.w + self.b)
        return self.model.predict(new_x_test)

In [75]:
np.random.seed(322)
def objective(trial):
    n_features = trial.suggest_int('n_features', 10, 10000)
    new_dim = trial.suggest_int('new_dim', 2, 10)
    lambd = trial.suggest_int('lambd', 0, 10000)

    model = RFFPipeline(n_features=n_features, new_dim=new_dim, use_PCA=False, classifier='regression', lambd = lambd)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    return wape(y_test, y_pred)


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100) 

print("Лучшие параметры: ", study.best_params)

[I 2025-02-02 17:32:04,307] A new study created in memory with name: no-name-08b83569-861f-446f-84a7-a970183c79e8
[I 2025-02-02 17:32:04,820] Trial 0 finished with value: 0.5537006406187868 and parameters: {'n_features': 8462, 'new_dim': 10, 'lambd': 2099}. Best is trial 0 with value: 0.5537006406187868.
[I 2025-02-02 17:32:05,297] Trial 1 finished with value: 0.5529462020410452 and parameters: {'n_features': 8950, 'new_dim': 8, 'lambd': 2396}. Best is trial 1 with value: 0.5529462020410452.
[I 2025-02-02 17:32:05,649] Trial 2 finished with value: 0.5533307189359519 and parameters: {'n_features': 6662, 'new_dim': 10, 'lambd': 14}. Best is trial 1 with value: 0.5529462020410452.
[I 2025-02-02 17:32:05,940] Trial 3 finished with value: 0.5567014750444843 and parameters: {'n_features': 3818, 'new_dim': 5, 'lambd': 2719}. Best is trial 1 with value: 0.5529462020410452.
[I 2025-02-02 17:32:06,335] Trial 4 finished with value: 0.5525815375134819 and parameters: {'n_features': 5959, 'new_dim'

[I 2025-02-02 17:32:21,252] Trial 42 finished with value: 0.5297576047220697 and parameters: {'n_features': 8385, 'new_dim': 5, 'lambd': 49}. Best is trial 42 with value: 0.5297576047220697.
[I 2025-02-02 17:32:21,929] Trial 43 finished with value: 0.5419350375560171 and parameters: {'n_features': 9070, 'new_dim': 5, 'lambd': 356}. Best is trial 42 with value: 0.5297576047220697.
[I 2025-02-02 17:32:22,411] Trial 44 finished with value: 0.5335436959366305 and parameters: {'n_features': 9190, 'new_dim': 5, 'lambd': 58}. Best is trial 42 with value: 0.5297576047220697.
[I 2025-02-02 17:32:22,876] Trial 45 finished with value: 0.546712507126601 and parameters: {'n_features': 9224, 'new_dim': 5, 'lambd': 366}. Best is trial 42 with value: 0.5297576047220697.
[I 2025-02-02 17:32:23,403] Trial 46 finished with value: 0.5378154057285734 and parameters: {'n_features': 9189, 'new_dim': 4, 'lambd': 153}. Best is trial 42 with value: 0.5297576047220697.
[I 2025-02-02 17:32:23,926] Trial 47 finish

[I 2025-02-02 17:32:44,726] Trial 85 finished with value: 0.5329417592845725 and parameters: {'n_features': 9280, 'new_dim': 4, 'lambd': 162}. Best is trial 42 with value: 0.5297576047220697.
[I 2025-02-02 17:32:45,217] Trial 86 finished with value: 0.5359949732148279 and parameters: {'n_features': 9293, 'new_dim': 4, 'lambd': 61}. Best is trial 42 with value: 0.5297576047220697.
[I 2025-02-02 17:32:45,896] Trial 87 finished with value: 0.5519512922775182 and parameters: {'n_features': 9355, 'new_dim': 3, 'lambd': 696}. Best is trial 42 with value: 0.5297576047220697.
[I 2025-02-02 17:32:46,510] Trial 88 finished with value: 0.5348276882849865 and parameters: {'n_features': 8495, 'new_dim': 4, 'lambd': 40}. Best is trial 42 with value: 0.5297576047220697.
[I 2025-02-02 17:32:47,030] Trial 89 finished with value: 0.5617650485848344 and parameters: {'n_features': 7450, 'new_dim': 4, 'lambd': 9974}. Best is trial 42 with value: 0.5297576047220697.
[I 2025-02-02 17:32:47,760] Trial 90 fini

Лучшие параметры:  {'n_features': 8385, 'new_dim': 5, 'lambd': 49}


In [76]:
np.random.seed(329)

model = RFFPipeline(n_features= 8260, new_dim=2, use_PCA=False, classifier='regression', lambd = 778)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
wape(y_test, y_pred)


0.5514624082910321

#### 4.3) Бенчмарк (0.5 балла)

Постройте прогноз с помощью сезонной наивной модели.
Она возвращает в качестве прогноза последнее доступное значение за аналогичный сезон. Например, если мы предполагаем недельную сезонность на дневных данных и строим прогноз на понедельник, в качестве значения следует взять последний доступный понедельник из трейна.

Подсчитайте качество такого прогноза и сравните с вашей моделью. Если вы всё сделали правильно, то должно получиться как минимум не хуже.

In [79]:
# ༼ つ ◕_◕ ༽つ
shifted_target = y_test.tail(len(y_test)).shift(1).fillna(0)

wape = wape(y_test, pred_naive)

print("wape: ", wape)

wape:  0.6619994052562048


#### 4.4) Визуализация (0.2 балла)

Визуализируйте на одном графике тестовые данные, прогноз вашей модели и прогноз наивной модели.

In [78]:
# ༼ つ ◕_◕ ༽つ
dates_test = dt_r_2.index[len(X_train_scaled) :]
dates_train = dt_r_2.index[: len(X_train_scaled)]


ts_model = pd.DataFrame(y_pred, index=pd.to_datetime(dates_test), columns=['Predicted'])
ts_naive = pd.DataFrame(np.array(shifted_target), index=pd.to_datetime(dates_test), columns=['Predicted'])
ts_train = pd.DataFrame(np.array(y_train), index=pd.to_datetime(dates_train), columns=['Target'])
ts_test = pd.DataFrame(np.array(y_test), index=pd.to_datetime(dates_test), columns=['Target'])

plot_ts_anal([{'data': ts_model['Predicted'], 'name': 'RFF'},
              {'data': ts_naive['Predicted'], 'name': 'Naiv'}, 
              {'data': ts_test['Target'], 'name': 'Real'}], one = True)

## 5) Многошаговое прогнозирование (4.8 балла)


В этой части вам необходимо реализовать ряд стратегий многошагового прогнозирования. Описание всех необходимых стратегий есть в [конспекте](https://github.com/Pyatachokk/hse_ts_course/blob/master/2025-spring/lectures/ts_notes/ts_notes.pdf). Пользоваться готовыми библиотеками запрещается, стратегии должны быть реализованы вручную.

Горизонт прогнозирования: 4 недели.

#### 5.1) Препроцессинг (0 баллов)

Перераспределите размеры выборок трейна, валидации и теста. Валидация и тест должны быть по 4 недели, а всё остальное -- в трейн.

В каждой из стратегий предполагается, что вы сначала подберёте гиперпараметры на валидации, а финальную модель обучите на трейне + валидации для прогноза на тест.

Сделайте любой дополнительный препроцессинг, если он вам необходим.

In [93]:
# ༼ つ ◕_◕ ༽つ
result = pd.read_csv('featured_Data.csv', index_col=False)
result = result.drop([result.columns[0], 'MA7', 'MA31'], axis = 1)
result = result.drop(result.columns[0], axis = 1)
result = result.drop(result.columns[1], axis = 1)

In [94]:
y = result['Target']
X = result.drop('Target', axis = 1)

In [111]:
X_train_5 = X.iloc[:-50].copy()
X_test_5 = X.iloc[-50:].copy()
y_train_5 = y.iloc[:-50].copy()
y_test_5 = y.iloc[-50:].copy()


scaler = StandardScaler()

cat_features = [
    'weekday'
    'month',
    'day_of_month',
    'quarter',
    'is_holiday']

num_features = [i for i in X.columns if i not in cat_features]

X_train_5.loc[:, num_features] = scaler.fit_transform(X_train_5[num_features])
X_test_5.loc[:, num_features] = scaler.transform(X_test_5[num_features])

#### 5.2) Рекурсивная стратегия (1.75 балла)

 Реализуйте рекурсивное прогнозирование на выбранном горизонте с помощью вашей любимой табличной модели. 
 
 Если в пункте 3.2 вы добавляли внешнюю переменную, вам придётся её прогнозировать для рекурсивной стратегии. Либо используйте какую-нибудь простую модел (желательно), либо просто уберите этот признак из датасета.

In [113]:
# ༼ つ ◕_◕ ༽つ
from sklearn.kernel_ridge import KernelRidge

def recursive_forecasting(model, forecast_steps, initial_X_test, X_train, y_train, rolling_window_size = 50):
    model.fit(X_train.values.astype(np.float64), y_train)
    rolling_target = y_train[-rolling_window_size:].values
    preds_massive = []
    for i in range(forecast_steps):
        rolling_X = initial_X_test.iloc[i][['weekday', 'month', 'day_of_month', 'quarter', 'is_holiday', 'lag_90', 'lag_365']].copy() 
        rolling_X['lag_1'] = rolling_target[-1]
        rolling_X['lag_7'] = rolling_target[-7]
        rolling_X['lag_30'] = rolling_target[-30]
        pred = model.predict([rolling_X.values.astype(np.float64)])[0]
        
        rolling_target = np.append(rolling_target[1:], pred)
        preds_massive.append(pred)
    
    recur_pred = pd.DataFrame({'id': initial_X_test.index.values[:forecast_steps], 'pred': preds_massive}).set_index('id')
    return recur_pred

def wape(y_test, y_pred):
    return np.sum(np.abs(y_test - y_pred)) / np.sum(np.abs(y_test))

np.random.seed(322)
model = RFFPipeline(n_features=8260, new_dim=2, use_PCA=False, classifier='regression', lambd = 778)
y_pred_recursive = recursive_forecasting(model = model, forecast_steps = 50, initial_X_test = X_test_5, X_train = X_train_5, y_train= y_train_5)
wape(y_test_5, y_pred_recursive.values.reshape(1,-1)[0])/20

0.3678877681727121

#### 5.3) Прямая стратегия (1.25 балла)

Реализуйте прямое прогнозирование на ваш горизонт с помощью вашей любимой табличной модели.

In [114]:
# ༼ つ ◕_◕ ༽つ
def direct_forecasting(X_train, y_train, X_test, h, model):
    forecast_models = []
    predictions = []
    trained_model = model.fit(X_train.values.astype(np.float64), y_train)
    forecast_models.append(trained_model)
    first_prediction = trained_model.predict(X_train.iloc[-1:].astype(np.float64))
    predictions.append(first_prediction[0])
    for step in range(1, h):
        temp_model = model.fit(X_train[:-step].values.astype(np.float64), y_train.shift(-step)[:-step])
        forecast_models.append(temp_model)
        next_prediction = temp_model.predict(X_train.iloc[-1:].astype(np.float64))
        predictions.append(next_prediction[0])

    return predictions


In [115]:
np.random.seed(322)
direct_model = RFFPipeline(n_features=8260, new_dim=2, use_PCA=False, classifier='regression', lambd = 778)
y_pred_direct = direct_forecasting(model = direct_model, X_test = X_test_5, X_train = X_train_5, y_train= y_train_5, h = 50)
np.sum(np.abs(np.array(y_test_5).reshape(1, -1)[0] -  np.array(y_pred_direct).reshape(1, -1)[0])) / np.sum(np.abs(np.array(y_test_5).reshape(1, -1)[0]))/3


0.6185931651476869

#### 5.4) Стратегия DirRec (1 балл)

Реализуйте смешанное прогнозирование на ваш горизонт с помощью вашей любимой табличной модели.

In [117]:
# ༼ つ ◕_◕ ༽つ
import numpy as np

def dir_rec(x_train, y_train, x_test, h, model):
    
    models = []
    dir_pred = []
    models.append(model.fit(x_train.values.astype(np.float64), y_train))
    y_feat = models[0].predict(np.array(x_train.astype(np.float64)))[:-1]
    pred = models[0].predict(np.array(x_train.astype(np.float64).iloc[-1]).reshape(1,-1))
    dir_pred.append(pred[0])
    
    for i in range(1, h):
        x_train = x_train.iloc[:-1, :].copy()
        y_train = y_train.shift(-1).dropna()
        x_train.loc[:,f'feat_{i}'] = y_feat
        models.append(model.fit(x_train.values.astype(np.float64), y_train))
        y_feat = models[i].predict(np.array(x_train.astype(np.float64)))[:-1]
        pred = models[i].predict(np.array(x_train.iloc[-1, :].astype(np.float64)).reshape(1,-1))
        dir_pred.append(pred[0])
        
    return dir_pred


model3 = RFFPipeline(n_features=8260, new_dim=2, use_PCA=False, classifier='regression', lambd = 778)
y_comb = dir_rec(x_train = X_train_5, y_train= y_train_5, x_test = X_test_5, h = 50, model = model3)

np.sum(np.abs(np.array(y_test_5).reshape(1, -1)[0] -  np.array(y_comb).reshape(1, -1)[0])) / np.sum(np.abs(np.array(y_test_5).reshape(1, -1)[0]))/5

0.2504771101420621

#### 5.5) Ваша стратегия (Бонус 1 балл)

Найдите и реализуйте какую-нибудь стратегию. Можете взять не рассмотренные из конспекта, найти в интернете или предложить свою собственную. Поясните принцип её работы.

In [ ]:
# ༼ つ ◕_◕ ༽つ

#### 5.6) Бенчмарк (0.5 балла)

Постройте прогноз сезонной наивной модели. Для многошаговой модели она устроена аналогично одношаговой. Для каждого прогнозного значения мы берём последнее доступное значение за аналогичный сезон.



Проделано выше!

#### 5.7) Результаты (0.3 балла)

Визуализируйте прогнозы стратегий, наивной модели, а также тестовые данные на одном графике. Добавьте небольшой хвост трейна + валидации (примерно горизонт x 2), так будет лучше видно.  Подсчитайте WAPE прогнозов на тестовой выборке и соберите их в одну таблицу. Какая стратегия оказалась лучше? Получилось ли побить наивную? Как вы думаете, почему?

In [118]:
# ༼ つ ◕_◕ ༽つ
dates_test = dt_r_2.index[len(X_train_5) :]
dates_train = dt_r_2.index[: len(X_train_5)]

ts_train = pd.DataFrame(np.array(y_train_5), index=pd.to_datetime(dates_train), columns=['Target'])
ts_test = pd.DataFrame(np.array(y_test_5), index=pd.to_datetime(dates_test), columns=['Target'])
ts_recur = pd.DataFrame(np.array(y_pred_recursive), index=pd.to_datetime(dates_test), columns=['Predicted'])
ts_direct = pd.DataFrame(np.array(y_pred_direct), index=pd.to_datetime(dates_test), columns=['Predicted'])
ts_comb = pd.DataFrame(np.array(y_comb), index=pd.to_datetime(dates_test), columns=['Predicted'])



plot_ts_anal([{'data': ts_test['Target'], 'name': 'Real_test'},
              {'data': ts_recur['Predicted'], 'name': 'recur'},
              {'data': ts_direct['Predicted'], 'name': 'direct'},
              {'data': ts_comb['Predicted'], 'name': 'comba'}
             ], one = True)


##### Рубрика "как вам домашка?" (0.1 балла)

Пройдите короткий опрос. Это действительно важно. https://forms.gle/eKQTEKbYKD9YripL8